In [1]:
from citipy import citipy
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
from api_keys import weather_api_key
import random

In [2]:
range_lat = (-90.00, 90.00)
range_lng = (-180.00, 180.00)

city_dict = {}

# random.seed(0)

i = 1
ran_length = random.randint(500,600)

while len(city_dict) < 30: # change this after completion
    rand_lat = round(random.uniform(range_lat[0], range_lat[1]), 2)
    rand_lng = round(random.uniform(range_lng[0], range_lng[1]), 2)
    
    city = citipy.nearest_city(rand_lat, rand_lng)
    
    if city.city_name in city_dict:
        pass
    else:
        city_dict[city.city_name] = (i, city.country_code, rand_lat, rand_lng)
        i += 1    

# city_dict

In [4]:
url_base = "http://api.openweathermap.org/data/2.5/weather?"

weather_dict = {}

city = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []
city_na = []
for key in city_dict:
    
    url_query = f"{url_base}q={key}&appid={weather_api_key}"
    
    request = requests.get(url_query)
    weather_dict[key] = (request.json())
    
    try:
        city.append(weather_dict[key]["name"])
        lat.append(float(weather_dict[key]["coord"]["lat"]))
        lng.append(float(weather_dict[key]["coord"]["lat"]))
        max_temp.append(float(weather_dict[key]["main"]["temp_max"]))
        humidity.append(float(weather_dict[key]["main"]["humidity"]))
        cloudiness.append(int(weather_dict[key]["clouds"]["all"]))
        wind_speed.append(float(weather_dict[key]["wind"]["speed"]))
        country.append(weather_dict[key]["sys"]["country"])
        date.append(int(weather_dict[key]["dt"]))
        print(f"Request {str(city_dict[key][0]).rjust(3, '0')} of {len(city_dict)} | AVAILABLE:     COMPLETE [{request}] ---> {key}")
        
    except KeyError:
        city_na.append(key)
        print(f"Request {str(city_dict[key][0]).rjust(3, '0')} of {len(city_dict)} | NOT AVAILABLE: COMPLETE [{request}] ---> {key}")

ConnectionError: HTTPConnectionPool(host='api.openweathermap.org', port=80): Max retries exceeded with url: /data/2.5/weather?q=misratah&appid=1009f485083bd95e8715574b8bd7422e (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C7ECCE4978>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [ ]:
# Cities not found in OpenWeatherAPI
len_cities_na = len(city_na)

if len_cities_na == 0:
    print(f"All cities found in the OpenWeather database")
else:
    if len_cities_na == 1:
        print(f"The following city was not found in the OpenWeather database:\n")
    else:
        print(f"The following {len_cities_na} cities were not found in the OpenWeather database:\n")
    for index, element in enumerate(city_na):
        print(f"{index+1}.- {element}")

In [ ]:
city_df = pd.DataFrame({
    "City": city,
    "Latitude": lat,
    "Longitude": lng,
    "Max. Temperature (F)": max_temp,
    "Humidity (%)": humidity,
    "Cloudiness (%)": cloudiness,
    "Wind Speed (mph)": wind_speed,
    "Country": country,
    "Date": date
})

# Saving the df as a csv
city_df.to_csv("city_df.csv", encoding = "utf-8", index = False)
city_df

In [ ]:
# Statistics of the DataFrame
summary_sts = city_df.describe()
summary_sts


In [ ]:
# Removing cities where the humidity > 100%

if len(city_df["City"][city_df["Humidity (%)"] > 100]) != 0:
    print(f"The following city/cities was/were removed:\n")
    print(city_df["City"][city_df["Humidity (%)"] > 100])
    city_df = city_df.drop(city_df[city_df["Humidity (%)"] > 100].index).reset_index(drop = True)
else:
    print("Humidity inpsection successfully passed.")


In [ ]:
plot_labels = ["Max. Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind Speed (mph)"]

for label in plot_labels:
    city_df.plot.scatter("Latitude", label)